# NPI-3: NPI is not NULL

QA step description: check if there is any null in npi. Test will pass if no null appears.

Starting Author: Amy Jin (amy@careset.com)

Date: April 16th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connect to Parenthood server:

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml


# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) col_name:               column to test
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, the test will print out the number of failed NPIs.


def npi_3(db_name, table_name, col_name):
    
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        print ('Test file: {}.{}'.format(db_name, table_name))
        print ('\n')
        
        query = ('''
            SELECT *
            FROM {db}.{t1}
            WHERE {col1} IS NULL OR {col1} = ''
            LIMIT 1;
        '''.format(db = db_name, t1 = table_name, col1 = col_name))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))

        
        if not len(rows):
            print ("Npi-3 test result: PASS")
        else:
            print ("Npi-3 test result: FAIL" + '\n')
            print ("The count of rows with blank or NULL {} is:".format(col_name) + '\n')
            

            # MySQL query to get count of NPIs fail the test
            query = ('''
                SELECT COUNT({col1})
                FROM {db}.{t1}
                WHERE {col1} IS NULL OR {col1} = '';
            '''.format(db = db_name, t1 = table_name, col1 = col_name))

            cur.execute(query)
            
            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]))
                print ('\n')
            
            
        cur.close()
        connection.close()

## Test Example

In [3]:
npi_3('_amy', 'test_data_good', 'npi')

Test file: _amy.test_data_good


Npi-3 test result: PASS


In [4]:
npi_3('_amy', 'test_data_bad1', 'npi')

Test file: _amy.test_data_bad1


Npi-3 test result: FAIL

The count of rows with blank npi is:

96




In [3]:
npi_3('client_celgene',' HCP_Geographic_distribution_of_MM_Patients', '`Physician NPI Number`')

Test file: client_celgene. HCP_Geographic_distribution_of_MM_Patients


Npi-3 test result: PASS
